<a href="https://colab.research.google.com/github/cjryanwashere/cjryanwashere/blob/main/functional_discriminator_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import os

In [62]:
path = 'drive/MyDrive/DROIDS_Webserver/trainingData'

In [63]:
def load_Data(path, amount):
  array = []
  dir = os.listdir(path)
  
  if amount>len(dir):
    print("Error, amount is too high")
    return 

  for i in range(amount):
    print(i)
    x = os.path.join(path, dir[i])
    x = open(x, 'r')
    x = x.read()

    x = x.split("\n")
    x.pop(0)

    k = x
    x = []
    for i in k:
      v = i.split()
      x.append(v)
    


    array.append(x)
    

  return array

In [ ]:
array = load_Data(path, 500)

In [68]:
def prepare_fluctuation_Dataset(array):
  train = []
  labels = []

  val = []
  val_labels = []


  for i in array:
    labelset = []
    trainset = []

    valset = []
    val_labelset = []

    count = 0 
    for j in i:
      if count<15000:
        count += 1
        labelset.append(j[0])

        trainset.append(j[1])
      else:
        count+=1
        if count<20000:
          val_labelset.append(j[0])
          valset.append(j[1])

    val.append(valset)
    val_labels.append(val_labelset)

    
    train.append(trainset)
    labels.append(labelset)

  return train, labels, val, val_labels

In [69]:
train, labels, val, val_labels = prepare_fluctuation_Dataset(array)

In [70]:
print(np.shape(train),np.shape(labels),np.shape(val),np.shape(val_labels))

(500, 15000) (500, 15000) (500, 4999) (500, 4999)


In [8]:
import tensorflow as tf
import numpy as np

In [71]:
def convert_to_floats(array):
  result = []

  for i in array:
    p = []
    for j in i:
      x = float(j)
      p.append(x)
    result.append(p)
  
  return result

In [ ]:
train = convert_to_floats(train)
train

In [ ]:
labels = convert_to_floats(labels)
labels

In [ ]:
val = convert_to_floats(val)
val

In [75]:
val_labels = convert_to_floats(val_labels)

In [ ]:
val_labels

In [77]:
np.shape(train)

(500, 15000)

In [78]:
np.shape(labels)

(500, 15000)

In [17]:
def prepare_single_group_fluctuation_dataset(train, labels):
  xtrain = []
  ylabels = []
  for i in range(len(train[0])):
    groupfluct = []
    for j in range(len(train)):
      groupfluct.append(train[j][i])
    xtrain.append(groupfluct)
    ylabels.append(labels[0][i])
  
  return xtrain, ylabels

In [79]:
train, labels = prepare_single_group_fluctuation_dataset(train, labels)
print(np.shape(train), np.shape(labels))

(15000, 500) (15000,)


In [88]:
val, val_labels = prepare_single_group_fluctuation_dataset(val, val_labels)
print(np.shape(val), np.shape(val_labels))

(4999, 500) (4999,)


In [80]:
modelV1 = tf.keras.models.Sequential([
                                      tf.keras.layers.Dense(500, activation='relu'),
                                      tf.keras.layers.Dense(300),
                                      tf.keras.layers.Dropout(0.2),
                                      tf.keras.layers.Dense(100),
                                      tf.keras.layers.Dropout(0.2),
                                      tf.keras.layers.Dense(1)
])

In [81]:
np.shape(tf.expand_dims(train, axis=1))

TensorShape([15000, 1, 500])

In [82]:
modelV1(tf.expand_dims(train[0], axis=0))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.09546283]], dtype=float32)>

In [83]:
modelV1.compile(optimizer='adam',
                loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics = ['accuracy'])

In [84]:
np.shape(tf.expand_dims(labels, axis=-1))

TensorShape([15000, 1])

In [85]:
labels = np.expand_dims(labels, axis=-1)
train = tf.expand_dims(train, axis=1)

In [86]:
modelV1.fit(train, labels, epochs = 10)

Epoch 1/10
469/469 [==============================] - 5s 8ms/step - loss: 0.2007 - accuracy: 0.9023
Epoch 2/10
469/469 [==============================] - 4s 8ms/step - loss: 0.0408 - accuracy: 0.9897
Epoch 3/10
469/469 [==============================] - 4s 8ms/step - loss: 0.0452 - accuracy: 0.9838
Epoch 4/10
469/469 [==============================] - 4s 8ms/step - loss: 0.0157 - accuracy: 0.9947
Epoch 5/10
469/469 [==============================] - 4s 8ms/step - loss: 0.0104 - accuracy: 0.9959
Epoch 6/10
469/469 [==============================] - 4s 8ms/step - loss: 0.0184 - accuracy: 0.9927
Epoch 7/10
469/469 [==============================] - 4s 8ms/step - loss: 0.0038 - accuracy: 0.9987
Epoch 8/10
469/469 [==============================] - 4s 8ms/step - loss: 0.0125 - accuracy: 0.9959
Epoch 9/10
469/469 [==============================] - 4s 8ms/step - loss: 0.0060 - accuracy: 0.9979
Epoch 10/10
469/469 [==============================] - 4s 8ms/step - loss: 0.0056 - accuracy: 0.9980

In [89]:
val = tf.expand_dims(val, axis=1)
val_labels = np.expand_dims(val_labels, axis=-1)
print(np.shape(val), np.shape(val_labels))

(4999, 1, 500) (4999, 1)


In [90]:
modelV1.fit(val, val_labels)

157/157 [==============================] - 1s 8ms/step - loss: 7.7743e-05 - accuracy: 1.0000
